## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [463]:
import numpy as np
from numba import jit,njit,vectorize
import pandas as pd
import datetime
import line_profiler

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [602]:
@njit
def generate():
    return np.random.randint(0, 1001, 1_000_000)

In [603]:
A = generate()
B = A+100
np.mean(B)

599.859088

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [604]:
def random_char():
    indexes = np.array([*range(ord('A'), ord('Z')+1), *range(ord('a'), ord('z')+1)])
    return list(map(chr, np.random.choice(indexes, 2000001)))

In [605]:
df = pd.DataFrame(data=np.random.randint(0, 1001, (2_000_001, 4)))
df['key'] = pd.Series(random_char(), index=df.index)
df

,0,1,2,3,key
0,318,164,551,79,k
1,619,637,415,962,N
2,854,529,566,8,Q
3,836,22,399,47,I
4,398,411,87,692,b
...,...,...,...,...,...
1999996,718,604,231,206,j
1999997,932,805,315,386,i
1999998,251,633,983,935,Z
1999999,440,694,33,21,e


In [606]:
df[(df.key == 'a')|(df.key[i] == 'b')|(df.key == 'c')|(df.key[i] == 'd')|(df.key == 'e')| \
  (df.key == 'A')|(df.key[i] == 'B')|(df.key == 'C')|(df.key[i] == 'D')|(df.key == 'E')]

,0,1,2,3,key
9,458,647,77,876,e
18,268,960,625,828,A
19,63,57,955,674,E
38,356,486,822,566,c
49,321,245,498,802,A
...,...,...,...,...,...
1999945,193,845,450,854,E
1999953,523,74,577,658,C
1999978,468,366,552,600,a
1999980,974,839,31,391,c


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [367]:
recipes = pd.read_csv('labs/recipes_sample.csv')
reviews = pd.read_csv('labs/reviews_sample.csv', index_col=0)

In [546]:
def mean1():
    a = 0
    for index, row in reviews.iterrows():
        a += row.rating
    return a/len(reviews.rating)

def mean2():
    reviews.date = pd.to_datetime(reviews.date)
    reviews2 = reviews[(reviews.date >= np.datetime64('2010-01-01')) & (reviews.date < np.datetime64('2011-01-01'))]
    
    a = 0
    for index, row in reviews2.iterrows():
        a += row.rating
    return a/len(reviews2.rating)

def mean3():
    return reviews.rating.mean()

In [493]:
print(f'Via iterrows: {mean1():.5f}\nVia iterrows (2010): {mean2():.5f}\nVia mean: {mean3():.5f}')

Via iterrows: 4.41080
Via iterrows (2010): 4.45444
Via mean: 4.41080


In [494]:
lp = LineProfiler()
lp_wrapper = lp(mean1)()
lp.print_stats()

Timer unit: 1e-09 s

Total time: 5.30871 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/3782130813.py
Function: mean1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean1():
     2         1       1000.0   1000.0      0.0      a = 0
     3    126696 4134938000.0  32636.7     77.9      for index, row in reviews.iterrows():
     4    126696 1173754000.0   9264.3     22.1          a += row.rating
     5         1      19000.0  19000.0      0.0      return a/len(reviews.rating)



In [495]:
lp = LineProfiler()
lp(mean2)()
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0.497533 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/3782130813.py
Function: mean2 at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def mean2():
     8         1    4623000.0 4623000.0      0.9      reviews.date = pd.to_datetime(reviews.date)
     9         1    2031000.0 2031000.0      0.4      reviews2 = reviews[(reviews.date >= np.datetime64('2010-01-01')) & (reviews.date < np.datetime64('2011-01-01'))]
    10                                               
    11         1          0.0      0.0      0.0      a = 0
    12     12094  379670000.0  31393.3     76.3      for index, row in reviews2.iterrows():
    13     12094  111165000.0   9191.7     22.3          a += row.rating
    14         1      44000.0  44000.0      0.0      return a/len(reviews2.rating)



In [496]:
lp = LineProfiler()
lp(mean3)()
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0.000307 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/3782130813.py
Function: mean3 at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           def mean3():
    17         1     307000.0 307000.0    100.0      return reviews.rating.mean()



2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

# Timer unit: 1e-09 s

Total time: 5.33197 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/3782130813.py
Function: mean1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mean1():
     2         1       3000.0   3000.0      0.0      a = 0
     3    126696 4175868000.0  32959.7     78.3      for index, row in reviews.iterrows():
     4    126696 1156037000.0   9124.5     21.7          a += row.rating
     5         1      65000.0  65000.0      0.0      return a/len(reviews.rating)


Самая медленная ф-ция DataFrame.iterrows исходной таблицы (A)
Наиболее сильно на время выполнения влияет цикл, перебирающий все значения

In [536]:
def mean1_mod():
    a = reviews.rating.sum()
    b = len(reviews.rating)
    return a / b

In [537]:
lp = LineProfiler()
lp(mean1_mod)()
lp.print_stats()

Timer unit: 1e-09 s

Total time: 0.001322 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/2451321630.py
Function: mean1_mod at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean1_mod():
     2         1    1235000.0 1235000.0     93.4      a = reviews.rating.sum()
     3         1      78000.0  78000.0      5.9      b = len(reviews.rating)
     4         1       9000.0   9000.0      0.7      return a / b



3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [667]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [717]:
def get_word_reviews_count2(df):
    word_reviews = {}
    for i in df.review.dropna():
        for j in i.split(' '):
            try:
                word_reviews[j] += 1
            except:
                word_reviews[j] = 1
                
    return word_reviews

In [718]:
lp = LineProfiler()
lp(get_word_reviews_count2)(reviews)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 3.70017 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/356987905.py
Function: get_word_reviews_count2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count2(df):
     2         1       3000.0   3000.0      0.0      word_reviews = {}
     3    126679   78214000.0    617.4      2.1      for i in df.review.dropna():
     4   6792010 1066508000.0    157.0     28.8          for j in i.split(' '):
     5   6792010  661421000.0     97.4     17.9              try:
     6   6617066 1791640000.0    270.8     48.4                  word_reviews[j] += 1
     7    174944   20689000.0    118.3      0.6              except:
     8    174944   81693000.0    467.0      2.2                  word_reviews[j] = 1
     9                                                           
    10         1          0.0      0.0      0.0      return word_reviews



In [713]:
get_word_reviews_count2(reviews)['of']

109029

In [714]:
get_word_reviews_count(reviews)['of']

109029

In [634]:
lp = LineProfiler()
lp(get_word_reviews_count)(reviews)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 19.2364 s
File: /var/folders/02/6cj0f5qn4y30_4fdyfzy3vnh0000gn/T/ipykernel_818/2826575548.py
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1       5000.0   5000.0      0.0      word_reviews = {}
     3    126679 5388128000.0  42533.7     28.0      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 1199103000.0   9465.7      6.2          recipe_id, review = row['recipe_id'], row['review']
     5    126679  339873000.0   2682.9      1.8          words = review.split(' ')
     6   6792010  760505000.0    112.0      4.0          for word in words:
     7   6617066 1271745000.0    192.2      6.6              if word not in word_reviews:
     8    174944   41299000.0    236.1      0.2                  word_reviews[word] = []
     9   6792010 1984012000.0    292.1     10.3             

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4